In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
import os, sys

In [3]:
import pandas as pd

In [16]:
sys.path.append('..')

from pyMultiOmics.constants import *
from pyMultiOmics.mapping import Mapper
from pyMultiOmics.common import set_log_level_info

# Demonstration of pyMultiOmics

## Load the processed Zebrafish data from [1]

[1] [Rabinowitz, Jeremy S., et al. "Transcriptomic, proteomic, and metabolomic landscape of positional memory in the caudal fin of zebrafish." Proceedings of the National Academy of Sciences 114.5 (2017): E717-E726.](https://www.pnas.org/content/114/5/E717.short)

In [5]:
DATA_FOLDER = os.path.abspath(os.path.join('test_data', 'zebrafish_data'))
DATA_FOLDER

'C:\\Users\\joewa\\Work\\git\\pyMultiOmics\\notebooks\\test_data\\zebrafish_data'

Read transcriptomics data (identified by their gene ids)

In [6]:
gene_data = pd.read_csv(os.path.join(DATA_FOLDER, 'gene_data_combined.csv'), index_col='Identifier')
gene_design = pd.read_csv(os.path.join(DATA_FOLDER, 'gene_design.csv'), index_col='sample')

Read proteomics data

In [7]:
protein_data = pd.read_csv(os.path.join(DATA_FOLDER, 'protein_data.csv'), index_col='Uniprot')
protein_design = pd.read_csv(os.path.join(DATA_FOLDER, 'protein_design.csv'), index_col='sample')

Read metabolomics data

In [8]:
compound_data = pd.read_csv(os.path.join(DATA_FOLDER, 'compound_data_kegg.csv'), index_col='Identifier')
compound_design = pd.read_csv(os.path.join(DATA_FOLDER, 'compound_design.csv'), index_col='sample')

In [9]:
set_log_level_info()

1

## Create a mapping object

The mapping object uses Reactome to map the different biological entities in the data:
- Transcripts (or genes) are connected to the proteins they encode
- Proteins and compounds are connected to reactions they're involved in
- Reactions are connected to pathways

In [10]:
m = Mapper(DANIO_RERIO, metabolic_pathway_only=True) \
        .set_gene(gene_data, gene_design) \
        .set_protein(protein_data, protein_design) \
        .set_compound(compound_data, compound_design) \
        .build()

2021-02-05 17:08:47.305 | INFO     | pyMultiOmics.functions:reactome_mapping:72 - Mapping genes -> proteins
2021-02-05 17:08:50.533 | INFO     | pyMultiOmics.functions:reactome_mapping:77 - Mapping proteins -> reactions
2021-02-05 17:08:59.331 | INFO     | pyMultiOmics.functions:reactome_mapping:85 - Mapping compounds -> reactions
2021-02-05 17:09:01.313 | INFO     | pyMultiOmics.functions:reactome_mapping:91 - Mapping reactions -> pathways
2021-02-05 17:09:02.344 | INFO     | pyMultiOmics.functions:reactome_mapping:102 - Mapping reactions -> proteins
2021-02-05 17:09:11.304 | INFO     | pyMultiOmics.functions:reactome_mapping:109 - Mapping reactions -> compounds
2021-02-05 17:09:16.367 | INFO     | pyMultiOmics.functions:reactome_mapping:121 - Mapping proteins -> genes
2021-02-05 17:09:55.424 | INFO     | pyMultiOmics.mapping:_add_nodes:206 - Processing nodes: genes
2021-02-05 17:10:02.229 | INFO     | pyMultiOmics.mapping:_add_nodes:206 - Processing nodes: proteins
2021-02-05 17:10:0

## DE analysis

Run t-test

In [11]:
from pyMultiOmics.pipelines import *

In [20]:
data_type = GENOMICS
min_replace = SMALL

data_df = m.gene_df
design_df = m.gene_design

case = 'Distal'
control = 'Proximal'

In [21]:
wi = WebOmicsInference(data_df, design_df, data_type, min_value=min_replace)
result_df = wi.run_ttest(case, control)

2021-02-05 17:15:43.305 | INFO     | pyMultiOmics.pipelines:run_ttest:120 - t-test case is Distal, control is Proximal


In [22]:
result_df

,padj,log2FoldChange
ENSDARG00000000001,0.024460,0.715686
ENSDARG00000000002,0.514784,0.127781
ENSDARG00000000018,0.003630,-0.620668
ENSDARG00000000019,0.049351,0.349941
ENSDARG00000000068,0.396128,-0.113797
...,...,...
ENSDARG00000105304,1.000000,0.000000
ENSDARG00000105305,0.391027,0.253777
ENSDARG00000105306,0.628077,-0.350978
ENSDARG00000105307,0.170340,-0.998356


Update the results back to graph

In [23]:
display_name = 't-test: %s_vs_%s' % (case, control)
display_name

't-test: Distal_vs_Proximal'